In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import cv2
import torch
from torchvision import transforms
from model import CNN  # Assuming you have already defined your CNN model

class TumorDetectionApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Brain Tumor Detection App")

        self.model = CNN()  # Instantiate your model here
        self.model.load_state_dict(torch.load('model.pth'))  # Load trained model weights

        self.label = tk.Label(master, text="Select an MRI image:")
        self.label.pack()

        self.load_button = tk.Button(master, text="Load Image", command=self.load_image)
        self.load_button.pack()

        self.result_label = tk.Label(master, text="")
        self.result_label.pack()

        self.image_label = tk.Label(master)
        self.image_label.pack()

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            image = self.preprocess_image(file_path)
            prediction = self.predict(image)
            self.display_result(prediction)
            self.display_image(file_path)

    def preprocess_image(self, file_path):
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (128, 128))
        image = np.transpose(image, (2, 0, 1))
        image = image.astype(np.float32) / 255.0
        image = torch.tensor(image)
        image = image.unsqueeze(0)
        return image

    def predict(self, image):
        with torch.no_grad():
            self.model.eval()
            prediction = self.model(image)
            prediction = torch.sigmoid(prediction)
            return prediction.item()

    def display_result(self, prediction):
        if prediction >= 0.5:
            result = "Tumor detected"
        else:
            result = "No tumor detected"
        self.result_label.config(text=result)

    def display_image(self, file_path):
        image = Image.open(file_path)
        image.thumbnail((400, 400))
        photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=photo)
        self.image_label.image = photo

def main():
    root = tk.Tk()
    app = TumorDetectionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


/Users/Nayyar/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'model'